# 因子分析 Factor Analysis

当有一组来自多个高斯分布的混合数据 $x^{(i)} \in \mathbb{R}^n$ 时，EM算法可以用来拟合高斯混合模型。这个问题中，通常假设拥有足够的数据，可以描绘出多个高斯分布的结构，严格来说，需要训练集大小 $m$ 显著地大于 数据维度 $n$。

现在，考虑 $n \gg m$ 的情况。这时，即便要拟合单个高斯模型，都是十分困难的，更不必说高斯混合模型。具体来看，由于 $m$ 个数据点稀疏地散布在较低的 $n$ 维空间中，假如要用高斯建模，并用最大似然估计法，估计均值和协方差，可得
$$
\begin{split}
\mu &= \frac{1}{m}\sum_{i=1}^m x^{(i)} \\
\Sigma &= \frac{1}{m}\sum_{i=1}^m (x^{(i)} - \mu)(x^{(i)} - \mu)^T
\end{split}
$$
可以发现，求出的结果，$\Sigma$ 是奇异矩阵，这意味着 $\Sigma^{-1}$ 不存在，并且 $1/|\Sigma|^{1/2}$ = 1/0，而这两项都需要用来计算多元高斯分布的概率密度。另一个困难点在于，最大似然估计法估计出的参数，会使得高斯分布的概率分布在数据的远交空间内。

更广泛来看，除非 $m$ 比 $n$ 大出较多的数量，否则最大似然估计法估计出的参数会非常糟糕。然而，我们依然希望对数据给出高斯模型的比较好的拟合，甚至捕捉到数据中关于协方差的一些结构信息。这要怎么做？

接下来，我们会首先增加两种可能的针对协方差的约束，使得可以用更小的数据集拟合 $\Sigma$。但这两种方法都无法给出令人满意的解。于是，我们会引入一些关于高斯分析的特性——边缘分布和条件分布——这将用于构建因子分析模型，以及对应的EM算法。

本节包含以下内容：
1. 约束协方差 $\Sigma$  Restrictions of $\Sigma$
2. 高斯分布的边缘分布和条件分布 Marginals and conditionals of Gaussians
3. 因子分析模型 The Factor analysis model
4. 在因子分析中应用EM算法 EM for Factor Analysis


### 1. 约束协方差 $\Sigma$  Restrictions of $\Sigma$

当没有足够的数据来拟合完整的协方差矩阵，可以考虑给协方差增加一些约束，从而减少对数据量的要求。比如，约束协方差为对角矩阵。在这个约束下，最大似然估计法估计出的协方差矩阵 $\Sigma$ 满足
$$ \Sigma_{jj} = \frac{1}{m}\sum_{i=1}^m (x_j^{(i)} - \mu_j)^2 $$
因此，$\Sigma_{jj}$ 是数据第 $j$ 坐标方差的经验估计。

回想高斯密度的轮廓图是椭圆形的。一个是对角矩阵的协方差矩阵，对应着所有对称轴都和坐标轴平行的椭圆。

有时，还可以进一步约束协方差矩阵不仅为对角矩阵，并且对角线上的所有值都相等，也就是说 $\Sigma = \sigma^2I$，这里 $\sigma^2$ 可以根据最大似然估计法获得
$$ \sigma^2 = \frac{1}{mn}\sum_{j=1}^n \sum_{i=1}^m (x_j^{(i)} - \mu_j)^2 $$

这个模型对应着轮廓图为圆形的高斯密度函数。（在超过二维的情形下，则是三维的球体，或者高维的超球面）

如果要用最大似然估计法拟合无约束的完整协方差矩阵 $\Sigma$，$\Sigma$ 可逆（不是奇异矩阵）的必要条件为 $m \geq n+1$。而引入上面两种约束的任意一种，则只需要 $m \geq 2$ 即可。

但是，约束 $\Sigma$ 为对角矩阵，也就意味着不等坐标的 $x_i$ 和 $x_j$ 相互独立不相关。更多的时候，我们还是希望尽可能地捕捉到数据维度的相关性信息。而因子分析模型，就是为了无法拟合整个协方差矩阵的前提下，来描述数据维度的相关性信息。